# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [30]:
import numpy as np

In [31]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [32]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.97143173659693, 10.0)

In [33]:
# Put your code here to price the corporate bond
def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    #Step 0: Generate exponential random numbers
    U = np.random.uniform(size=n_sample)
    default_time = -(1 / def_rate) * np.log(U)
    
    #Step 1: Calculate the price for each default time
    price_list = []
    for i in default_time:
        if i <= mat:
            price = recovery * 1.0 * np.exp(-rf_rate * i)
        else:
            price = 1.0 * np.exp(-rf_rate * mat)
        price_list.append(price)
    
    #Step 2: Calculate the mean of all the prices
    price_array = np.array(price_list)
    price_mean = np.mean(price_array)
    return price_mean

# Call your function
price = corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
print('Result #0: The result of calling my function is', price)

# Find the mean and std by calling the function 100 times. 
iterations = 100
price_list = []
for j in range(iterations):
    price = corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
    price_list.append(price)

price_mean = np.mean(price_list)
price_std = np.std(price_list)
print('Result #1: After calling the function 100 times, \
the mean is {0}, and the std is {1}.'.format(price_mean, price_std))

Result #0: The result of calling my function is 0.44061475978976833
Result #1: After calling the function 100 times, the mean is 0.4400637534580529, and the std is 0.0024609039741320614.


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [34]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [35]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    ### <--
    if(antithetic):
        U = np.random.uniform(size=int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else:
        U = np.random.uniform(size=n_sample)
        default_time = -(1 / def_rate) * np.log(U)
    
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
    
    #Calculate the price as above
    price_list = []
    for i in default_time:
        if i <= mat:
            price = recovery * 1.0 * np.exp(-rf_rate * i)
        else:
            price = 1.0 * np.exp(-rf_rate * mat)
        price_list.append(price)
    price_mean = np.mean(price_list)
    return price_mean

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
iterations = 100

#(i)antithetic
antithetic = 1
mean_match = 0
price_list = []
for j in range(iterations):
    price = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,\
                        antithetic=antithetic, mean_match=mean_match)
    price_list.append(price)

price_mean = np.mean(price_list)
price_std = np.std(price_list)
print('Result #2: Only using antithetic method, \
the mean is {0}, and the std is {1}.'.format(price_mean, price_std))


#(ii)mean_match
antithetic = 0
mean_match = 1
price_list = []
for j in range(iterations):
    price = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,\
                        antithetic=antithetic, mean_match=mean_match)
    price_list.append(price)

price_mean = np.mean(price_list)
price_std = np.std(price_list)
print('Result #3: Only using mean_match method, \
the mean is {0}, and the std is {1}.'.format(price_mean, price_std))

#(ii)both
antithetic = 1
mean_match = 1
price_list = []
for j in range(iterations):
    price = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,\
                        antithetic=antithetic, mean_match=mean_match)
    price_list.append(price)

price_mean = np.mean(price_list)
price_std = np.std(price_list)
print('Result #4: Using both antithetic method and mean_match method, \
the mean is {0}, and the std is {1}.'.format(price_mean, price_std))

Result #2: Only using antithetic method, the mean is 0.44061965674555414, and the std is 0.001450520603925647.
Result #3: Only using mean_match method, the mean is 0.4405781949948064, and the std is 0.0016522257259707174.
Result #4: Using both antithetic method and mean_match method, the mean is 0.44034398302802524, and the std is 0.0013666915666453961.


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [29]:
### Put the analytic expression for the corporate bond price
r = def_rate + rf_rate
price_analytic = 0.3 * (1 - np.exp(-r*mat)) * def_rate / r + np.exp(-r * mat)
print('Result #5: The analytic price is', price_analytic)
print('Result #6: The MC result using both antithetic method and mean_match \
method(same as ) is', price_mean)
print('Result #7: The difference between them is', price_analytic - price_mean)

The analytic price is 0.44040907156462505
The MC result using both antithetic method and mean_match method is 0.4403549291506378
The difference between them is 5.4142413987268156e-05
